In [9]:
import pandas as pd
import numpy as np
import json
import torch
import time

Install spotlight packages

In [2]:
!pip install --user git+https://github.com/maciejkula/spotlight.git@master#egg=spotlight

In [3]:
!pip install --user --upgrade pip

Requirement already up-to-date: pip in /users/msc/2413917c/.local/lib/python3.6/site-packages (19.2.1)


Dataset could be downloaded from here: https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Electronics_v1_00.tsv.gz 

In [2]:
df = pd.read_csv('amazon_reviews_us_Electronics_v1_00.tsv.gz', delimiter='\t',error_bad_lines=False)
df.head()

b'Skipping line 9076: expected 15 fields, saw 22\nSkipping line 19256: expected 15 fields, saw 22\nSkipping line 24313: expected 15 fields, saw 22\nSkipping line 47211: expected 15 fields, saw 22\nSkipping line 54295: expected 15 fields, saw 22\nSkipping line 56641: expected 15 fields, saw 22\nSkipping line 63067: expected 15 fields, saw 22\n'
b'Skipping line 93796: expected 15 fields, saw 22\n'
b'Skipping line 132806: expected 15 fields, saw 22\nSkipping line 164631: expected 15 fields, saw 22\nSkipping line 167019: expected 15 fields, saw 22\nSkipping line 167212: expected 15 fields, saw 22\n'
b'Skipping line 198103: expected 15 fields, saw 22\nSkipping line 199191: expected 15 fields, saw 22\nSkipping line 202841: expected 15 fields, saw 22\nSkipping line 218228: expected 15 fields, saw 22\nSkipping line 235900: expected 15 fields, saw 22\n'
b'Skipping line 277761: expected 15 fields, saw 22\nSkipping line 304582: expected 15 fields, saw 22\nSkipping line 312029: expected 15 fields,

,marketplace,customer_id,review_id,product_id,product_parent,product_title,product_category,star_rating,helpful_votes,total_votes,vine,verified_purchase,review_headline,review_body,review_date
0,US,41409413,R2MTG1GCZLR2DK,B00428R89M,112201306,yoomall 5M Antenna WIFI RP-SMA Female to Male ...,Electronics,5,0,0,N,Y,Five Stars,As described.,2015-08-31
1,US,49668221,R2HBOEM8LE9928,B000068O48,734576678,"Hosa GPM-103 3.5mm TRS to 1/4"" TRS Adaptor",Electronics,5,0,0,N,Y,It works as advertising.,It works as advertising.,2015-08-31
2,US,12338275,R1P4RW1R9FDPEE,B000GGKOG8,614448099,Channel Master Titan 2 Antenna Preamplifier,Electronics,5,1,1,N,Y,Five Stars,Works pissa,2015-08-31
3,US,38487968,R1EBPM82ENI67M,B000NU4OTA,72265257,LIMTECH Wall charger + USB Hotsync & Charging ...,Electronics,1,0,0,N,Y,One Star,Did not work at all.,2015-08-31
4,US,23732619,R372S58V6D11AT,B00JOQIO6S,308169188,Skullcandy Air Raid Portable Bluetooth Speaker,Electronics,5,1,1,N,Y,Overall pleased with the item,Works well. Bass is somewhat lacking but is pr...,2015-08-31


In [61]:
df1 = df.copy()
df1.count()

marketplace          3091024
customer_id          3091024
review_id            3091024
product_id           3091024
product_parent       3091024
product_title        3091020
product_category     3091024
star_rating          3091024
helpful_votes        3091024
total_votes          3091024
vine                 3091024
verified_purchase    3091024
review_headline      3090993
review_body          3090936
review_date          3091000
dtype: int64

In [62]:
df1 = df1[df1.review_date.notnull()]
df1 = df1[df1.product_title.notnull()]
df1 = df1[df1.product_id.notnull()]
df1 = df1[df1.customer_id.notnull()]
df1.count()

marketplace          3090996
customer_id          3090996
review_id            3090996
product_id           3090996
product_parent       3090996
product_title        3090996
product_category     3090996
star_rating          3090996
helpful_votes        3090996
total_votes          3090996
vine                 3090996
verified_purchase    3090996
review_headline      3090965
review_body          3090908
review_date          3090996
dtype: int64

In [5]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3090996 entries, 0 to 3091023
Data columns (total 15 columns):
marketplace          object
customer_id          int64
review_id            object
product_id           object
product_parent       int64
product_title        object
product_category     object
star_rating          int64
helpful_votes        int64
total_votes          int64
vine                 object
verified_purchase    object
review_headline      object
review_body          object
review_date          object
dtypes: int64(5), object(10)
memory usage: 377.3+ MB


In [63]:
df1['review_date'] = pd.to_datetime(df1['review_date']).dt.strftime("%Y%m%d")

In [65]:
min_r = 1850
min_c = 5
num_review = df1[['product_id']].groupby(df1['product_id']).count()
num_customer = df1[['customer_id']].groupby(df1['customer_id']).count()
print(len(np.where(num_review>min_r)[0]))
# print(len(num_review))
print(len(np.where(num_customer>min_c)[0]))
# a = np.where(num_review>min_r)[0] 
# b = np.where(num_customer>min_c)[0]
customer = num_customer.index[np.where( df1[['customer_id']].groupby(df1['customer_id']).count()>min_c)[0]]
# print(len(customer))
item = num_review.index[np.where(df1[['product_id']].groupby(df1['product_id']).count()>min_r)[0]]
# print(len(item))

100
34038


In [66]:
start = time.time()
df2 =  pd.DataFrame( columns = df1.columns) 
for i in range(len(item)):
    df2 = df2.append(df1[df1['product_id']==item[i]])
end = time.time()
diff = end-start
print('Data processing cost %d seconds' % (diff))

Data processing cost 80 seconds


In [67]:
df2.count()

marketplace          347053
customer_id          347053
review_id            347053
product_id           347053
product_parent       347053
product_title        347053
product_category     347053
star_rating          347053
helpful_votes        347053
total_votes          347053
vine                 347053
verified_purchase    347053
review_headline      347051
review_body          347048
review_date          347053
dtype: int64

In [68]:
print(df2[['star_rating']].groupby(df2['star_rating']).count())
np.min(df2[['star_rating']].groupby(df2['star_rating']).count())

             star_rating
star_rating             
1                  23485
2                  14791
3                  21747
4                  55765
5                 231265


star_rating    14791
dtype: int64

In [69]:
from spotlight.sampling import sample_items
df_g1 =  pd.DataFrame( columns = df1.columns) 
num_sample = np.min(df2[['star_rating']].groupby(df2['star_rating']).count())
for i in range(len(df2['star_rating'].unique())):
    df_g1 =  df_g1.append(df2[df2['star_rating']==i+1].sample(10000, random_state=1))
# df2

In [70]:
df_g1[['star_rating']].groupby(df_g1['star_rating']).count()

,star_rating
star_rating,
1,10000
2,10000
3,10000
4,10000
5,10000


In [71]:
df1 = df_g1

In [33]:
from spotlight.sampling import sample_items
df1 = df1.sample(30000, random_state=1)

In [72]:
# def frac(dataframe, fraction, other_info=None):
#     """Returns fraction of data"""
#     return dataframe.sample(frac=fraction)
# df1 = frac(df1, 0.1)
df1.count()

marketplace          50000
customer_id          50000
review_id            50000
product_id           50000
product_parent       50000
product_title        50000
product_category     50000
star_rating          50000
helpful_votes        50000
total_votes          50000
vine                 50000
verified_purchase    50000
review_headline      49998
review_body          49999
review_date          50000
dtype: int64

In [73]:
# print(df[['product_id']].groupby(df['product_id']).count())
# mean of goup by item
print(df1[['product_id']].groupby(df1['product_id']).count().mean())
# print(df[['customer_id']].groupby(df['customer_id']).count())
# mean of goup by user
print(df1[['customer_id']].groupby(df['customer_id']).count().mean())
print(df1[['star_rating']].astype('int').mean())

product_id    500.0
dtype: float64
customer_id    1.014755
dtype: float64
star_rating    3.0
dtype: float64


In [74]:
print(len(df1['product_id'].unique().tolist()))
print(len(df1['customer_id'].unique().tolist()))

100
49273


In [75]:
from collections import defaultdict
from itertools import count

uid_map = defaultdict(count().__next__)
iid_map = defaultdict(count().__next__)
uids = np.array([uid_map[uid] for uid in df1["customer_id"].values ], dtype=np.int32)
iids = np.array([iid_map[iid] for iid in df1["product_id"].values], dtype=np.int32)

uid_rev_map = {v: k for k, v in uid_map.items()}
iid_rev_map = {v: k for k, v in iid_map.items()}

ratings = df1["star_rating"].values.astype(np.float32)
timestamps = df1["review_date"].values.astype(np.object)

print("userId %d got uid %d" % (41409413, uid_map[41409413]))
print("itemId %s got iid %s" % ('B00428R89M', iid_map['B00428R89M']))

userId 41409413 got uid 49273
itemId B00428R89M got iid 100


In [76]:
from spotlight.cross_validation import user_based_train_test_split
from spotlight.datasets.synthetic import generate_sequential
from spotlight.evaluation import sequence_mrr_score
from spotlight.sequence.implicit import ImplicitSequenceModel
from spotlight.interactions import Interactions
from spotlight.cross_validation import random_train_test_split
imp_dataset = Interactions(user_ids=uids, item_ids=iids, num_users=len(uid_map), num_items=len(iid_rev_map))

In [77]:
## from spotlight.cross_validation import random_train_test_split

itrain, itest = random_train_test_split(imp_dataset, test_percentage=0.2,random_state=np.random.seed(42))
print(itrain)
print(itest)

<Interactions dataset (49274 users x 100 items x 40000 interactions)>
<Interactions dataset (49274 users x 100 items x 10000 interactions)>


In [84]:
from spotlight.factorization.implicit import ImplicitFactorizationModel
import time  

imodel = ImplicitFactorizationModel(loss='bpr',
                                    batch_size = 128,
                                    n_iter = 300, #bpr default but could take a long time
                                    embedding_dim = 64,
                                    l2=1e-9,  # strength of L2 regularization
                                    learning_rate=1e-3,)
current = time.time()

imodel.fit(itrain, verbose=True)
end = time.time()
diff = end - current
print("Training took %d seconds" % (diff))

Epoch 0: loss 0.49220143453762555
Epoch 1: loss 0.46912828344887436
Epoch 2: loss 0.40346176308184006
Epoch 3: loss 0.28457751460730457
Epoch 4: loss 0.17234759708753408
Epoch 5: loss 0.09797628978475595
Epoch 6: loss 0.05706079447040924
Epoch 7: loss 0.0358389283592899
Epoch 8: loss 0.024172786533975373
Epoch 9: loss 0.01791081481431715
Epoch 10: loss 0.014072929209949205
Epoch 11: loss 0.011576056034086991
Epoch 12: loss 0.010590606199476285
Epoch 13: loss 0.009065603246739783
Epoch 14: loss 0.007845594543225753
Epoch 15: loss 0.007530503095493625
Epoch 16: loss 0.0069895239142230906
Epoch 17: loss 0.006287467451736379
Epoch 18: loss 0.005869459454565288
Epoch 19: loss 0.006208436658826118
Epoch 20: loss 0.006000909570473642
Epoch 21: loss 0.006122304302173111
Epoch 22: loss 0.005474632050984869
Epoch 23: loss 0.005646059276185001
Epoch 24: loss 0.0054816739401592615
Epoch 25: loss 0.005250099275749141
Epoch 26: loss 0.0057411949855451
Epoch 27: loss 0.0052924305989076725
Epoch 28: l

Epoch 224: loss 0.005067498411257237
Epoch 225: loss 0.005395823021452076
Epoch 226: loss 0.004970701826444735
Epoch 227: loss 0.005036085338209765
Epoch 228: loss 0.005057926728619268
Epoch 229: loss 0.005286476016937258
Epoch 230: loss 0.004703882874879307
Epoch 231: loss 0.0050728164550762015
Epoch 232: loss 0.005385942655963639
Epoch 233: loss 0.0048752460396661165
Epoch 234: loss 0.005196078359104764
Epoch 235: loss 0.0050430031026157136
Epoch 236: loss 0.004906238303683436
Epoch 237: loss 0.005726120730760856
Epoch 238: loss 0.0050761130290290415
Epoch 239: loss 0.004926035826365216
Epoch 240: loss 0.005336795195627708
Epoch 241: loss 0.00560249381225568
Epoch 242: loss 0.004919940900331298
Epoch 243: loss 0.005277552234288603
Epoch 244: loss 0.004762022851255184
Epoch 245: loss 0.004972886198911423
Epoch 246: loss 0.0049443478121782266
Epoch 247: loss 0.00511426068009279
Epoch 248: loss 0.005399866011767342
Epoch 249: loss 0.005262793247073222
Epoch 250: loss 0.00558473078307871

Training - implicit model with BPR parameter --but not for iteration

In [15]:
import torch
torch.save(imodel, 'bpr_model')
bpr_model = torch.load("bpr_model")

In [16]:
bpr_model

<ImplicitFactorizationModel: BilinearNet(
  (user_embeddings): ScaledEmbedding(279789, 32)
  (item_embeddings): ScaledEmbedding(61286, 32)
  (user_biases): ZeroEmbedding(279789, 1)
  (item_biases): ZeroEmbedding(61286, 1)
)>

In [17]:
print(bpr_model.predict(0))
print(len(bpr_model.predict(0)))

[ 1.0979929   1.3855354   0.5076501  ... -0.26689515 -0.26815286
 -0.24741805]
61286


In [18]:
import scipy.stats as st

def tracksForUser(user, k=4):
    ranks = st.rankdata(-bpr_model.predict(user))  
    for iid in np.argwhere(ranks <= k):
        print(iid_rev_map[iid.item(0)])

tracksForUser(0)

B0002L5R78
B0012S4APK
B003L1ZYYM
B000WYVBR0


In [19]:
df1[df1["customer_id"]==uid_rev_map.get(1)].groupby("product_id")["product_id"].count()

product_id
B00H2QUD9S    1
Name: product_id, dtype: int64

Evaluation

In [20]:
def scoreAll(model):
    scoresForAllUsers=[]
    for uid in range(0,imodel._num_users):
        scoresForAllUsers.append(imodel.predict(uid))
    return scoresForAllUsers

In [21]:
import scipy.stats as st
#returns a numpy array of RR scores for each user.
#parameter: testInteractions -- a test interaction dataset
#parameter: predictions_for_each_user -- a list of numpy arrays of items scores for each user
#parameter: cutoff - a rank cutoff for the MRR user. Default is 0, which means no cutoff.
#parameter: train - the training data for the recommender - if set, it will not evaluate items that were part of the training data.
def calc_reciprank(testInteractions, predictions_for_each_user, train=None, cutoff=0):

    FLOAT_MAX = np.finfo(np.float32).max
    assert testInteractions.num_users == len(predictions_for_each_user), "mismatch: num users in Interactions %d, num users in predictions was %d" % (testInteractions.num_users,len(predictions_for_each_user))
    assert testInteractions.num_items == len(predictions_for_each_user[0]), "mismatch: num items in Interactions %d, num items in predictions was %d" % (testInteractions.num_items, len(predictions_for_each_user[0]))
  
    rrs = []
  
    if train is not None:
        train = train.tocsr()
    
    i=-1
    for user_id, row in enumerate(testInteractions.tocsr()):
        i=i+1
        if not len(row.indices):
            rrs.append(0)
            continue
      
        predictions = -predictions_for_each_user[i]
    
    #if train is set, remove the interactions from the training dataset
        if train is not None:
            predictions[train[user_id].indices] = FLOAT_MAX
    
        rank = st.rankdata(predictions)[row.indices].min()
    
    #we may have a rank cutoff
        if cutoff > 0 and rank > cutoff:
            rrs.append(0)
            continue
    
        rrs.append(1.0 / rank)
    return np.array(rrs)

ERROR happens since here

In [85]:
import time
from spotlight.evaluation import mrr_score
start = time.time()
mrr = mrr_score(imodel,itest)
end = time.time()
diff = end-start
print('Evaluation cost %d seconds'% (diff))

Evaluation cost 5 seconds


iteration: 300, dim: 64, 49274 users x 100 items, Training took 5545 seconds

In [86]:
np.mean(mrr)

0.09782865693671954

iteration: 200, dim: 64, 49274 users x 100 items, Training took 3937 seconds

In [83]:
np.mean(mrr)

0.101119066871709

iteration: 100, dim: 64, 49274 users x 100 items, Training took 2181 seconds

In [80]:
np.mean(mrr)

0.09788965582899394

iteration: 200, dim: 64, 97381 users x 247 items, Training took 14273 seconds

In [38]:
np.mean(mrr)

0.0598975876405622

iteration: 100, dim: 64, 97381 users x 247 items, Training took 8312 seconds

In [35]:
np.mean(mrr)

0.060146896149635774

In [ ]:
iteration: 100, dim: 64, 117493 users x 248 items, Training took 11421 seconds

In [55]:
np.mean(mrr)

0.06203551412119327

iteration = 1000, dim =32

In [33]:
np.mean(mrr)

0.02457670722678272

iteration = 1000, dim =64

In [36]:
np.mean(mrr)

0.02457670722678272

iteration = 500, dim = 64

In [30]:
np.mean(mrr)

0.04183405295243638

iteration = 200, dim = 32

In [25]:
import time
from spotlight.evaluation import mrr_score
start = time.time()
mrr = mrr_score(imodel,itest)
end = time.time()
diff = end-start
print('Evaluation cost %d seconds'% (diff))

Evaluation cost 6 seconds


In [27]:
np.mean(mrr)

0.033059801075278304

In [15]:
imodel

<ImplicitFactorizationModel: BilinearNet(
  (user_embeddings): ScaledEmbedding(532061, 32)
  (item_embeddings): ScaledEmbedding(87298, 32)
  (user_biases): ZeroEmbedding(532061, 1)
  (item_biases): ZeroEmbedding(87298, 1)
)>

In [16]:
import torch
torch.save(imodel, 'bpr_model_600000')
bpr_model = torch.load("bpr_model")

In [18]:
test1, test2 = random_train_test_split(itest, test_percentage=0.2,random_state=np.random.seed(42))

In [22]:
test2

<Interactions dataset (532061 users x 87298 items x 24000 interactions)>